# Loading libraries

In [ ]:
from typing import List, Union, Dict
import sys
import os
import glob
import datetime
import yaml
import warnings
sys.path.insert(1, '..')
os.chdir('..')

import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn
import optuna
import datetime

from darts import models
from darts import metrics
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler

from statsforecast.models import AutoARIMA

from data_formatter.base import *
from bin.utils import *

# Covariates processing

## Load Glucose data

In [ ]:
# Loop over the folder of each subject and merge files with insulin data by id
subject_ids = ["001", "002", "003", "004", "005", "006", "007", "008", "009"]

df_list = []
for subject_id in subject_ids:
    subject_data = pd.read_csv(f"raw_data/dubosson_covariates/diabetes_subset_pictures-glucose-food-insulin/{subject_id}/glucose.csv")
    subject_data["id"] = subject_id
    df_list.append(subject_data)

glucose_data = pd.concat(df_list, axis=0, ignore_index=True)
glucose_data

In [ ]:
# Create one daytime column 
glucose_data['date'] = pd.to_datetime(glucose_data['date'])
glucose_data['time'] = pd.to_datetime(glucose_data['time'], format='%H:%M:%S').dt.time
glucose_data['time'] = glucose_data.apply(lambda x: datetime.datetime.combine(x['date'], x['time']), axis=1)
# Keep the observations with cgm type only
glucose_data = glucose_data[(glucose_data['type']=='cgm')]
# Drop Date, Type, Comments columns
glucose_data.drop(["date", "type", "comments"], axis=1, inplace=True)
# Covert subject ids to int64 to match with "data" ids
glucose_data['id'] = glucose_data['id'].astype(int)
# Check for NaNs
glucose_data.isna().sum() # no NaN values
# Convert glucose readings from mmol/l to mg/dl
glucose_data['glucose'] = 18*glucose_data['glucose']
# rename Glucose column to gl
glucose_data.rename(columns={'glucose': 'gl'}, inplace=True)
# Reorder the columns
glucose_data = glucose_data[['id', 'time', 'gl']]
# reset index
glucose_data.reset_index(drop=True, inplace=True)

glucose_data

In [ ]:
# Plot glucose for each subject
groups = glucose_data.groupby('id')
num_cols = 3
num_rows = int(np.ceil(len(groups)/num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, (group_name, group) in enumerate(groups):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axs[row_idx, col_idx]
    ax.plot(group['time'], group['gl'])
    ax.set_xlabel('Time')
    ax.set_ylabel('Glucose')
    ax.set_title(f'Glucose Plot for ID {group_name}')

plt.tight_layout()
plt.show()

## Insulin covariates

In [ ]:
# Loop over the folder of each subject and merge files with insulin data by id
subject_ids = ["001", "002", "003", "004", "005", "006", "007", "008", "009"]

df_list = []
for subject_id in subject_ids:
    subject_data = pd.read_csv(f"raw_data/dubosson_covariates/diabetes_subset_pictures-glucose-food-insulin/{subject_id}/insulin.csv")
    subject_data["id"] = subject_id
    df_list.append(subject_data)

insulin_data = pd.concat(df_list, axis=0, ignore_index=True)
insulin_data

In [ ]:
# Create one daytime column 
insulin_data['date'] = pd.to_datetime(insulin_data['date'])
insulin_data['time'] = pd.to_datetime(insulin_data['time'], format='%H:%M:%S').dt.time
insulin_data['datetime'] = insulin_data.apply(lambda x: datetime.datetime.combine(x['date'], x['time']), axis=1)
# Drop Date, Time, Comment columns
insulin_data.drop(["date", "time", "comment"], axis=1, inplace=True)
# Covert subject ids to int64 to match with "data" ids
insulin_data['id'] = insulin_data['id'].astype(int)

insulin_data

In [ ]:
# Merge the two datasets based on "id"
df = insulin_data.merge(glucose_data, on='id')
# For each row in insulin_data, calculate the absolute difference
df['diff'] = (df['datetime'] - df['time']).abs()
# Find the index of the minimum difference for each subject and each insulin date-time
idx = df.groupby(['id', 'datetime'])['diff'].idxmin()
# Use that index to retrieve the corresponding "time" value
df_final = df.loc[idx, ['id', 'datetime', 'time']]
df_final.rename(columns={'id': 'id', 'time': 'closest_time'}, inplace=True)
# Add the closest time as a new column in insulin_data
result = insulin_data.merge(df_final, on=['id', 'datetime'], how='left')
# Calculate the difference between the closest time and datetime in minutes
result.loc[:, 'time_diff'] = np.abs((result['closest_time'] - result['datetime']) / np.timedelta64(1, 'm'))
# Keep only the rows where the absolute difference is less than or equal to 5 minutes
result = result.loc[result['time_diff'] <= 5, :]
# Some rows have exact the same closest_time when a person took fast and slow insulin at the same time. 
# Merge these duplicate rows in one row
result = result.groupby(["id", "closest_time"]).agg({"fast_insulin": "sum", "slow_insulin": "sum"}).reset_index()
# Merge glucose and insulin datasets
data_cov = glucose_data.merge(result, how='left', left_on=['id', 'time'], right_on=['id', 'closest_time'])
# Drop closest_time column
data_cov.drop(["closest_time"], axis=1, inplace=True)
# Replace observed values of zeroes with NaN
data_cov = data_cov.replace(0, np.nan)

data_cov

In [ ]:
## Prepare data for the interpolation

# set NaN right before observed values to 0 for slow_insulin
mask = data_cov['slow_insulin'].isna() & data_cov['slow_insulin'].shift(-1).notna()
data_cov.loc[mask, 'slow_insulin'] = 0
# set NaN right before observed values to 0 for fast_insulin
mask = data_cov['fast_insulin'].isna() & data_cov['fast_insulin'].shift(-1).notna()
data_cov.loc[mask, 'fast_insulin'] = 0

# Insert 0 instead of NaN in slow_insulin after approx. 24 hours after last observation 
# or at the very last observation for each subject
subset_data = []
for id in data_cov['id'].unique():
    # select only the rows for the current id
    id_data_cov = data_cov[data_cov['id']==id]
    # Find index of the last observed value
    last_observed_index = id_data_cov['slow_insulin'].last_valid_index()
    # If there are no observed values at all, insert 0 to the last observation of the subset
    if pd.isna(last_observed_index):
        slow_insulin = id_data_cov['slow_insulin'].copy()
        id_data_cov.loc[id_data_cov.index[-1], 'slow_insulin'] = 0 
    else:
        # Insert 0 instead of NaN in slow_insulin after approx. 24 hours after last valis observation
        id_data_cov.loc[last_observed_index + 228, 'slow_insulin'] = 0
    # Append the subjects' subsets
    subset_data.append(id_data_cov)
# Covert the data back to dataframe  
data_cov = pd.concat(subset_data, ignore_index=True)

# Insert 0 instead of NaN in fast_insulin after approx. 2 hours after last observation 
# or at the very last observation for each subject
subset_data = []
for id in data_cov['id'].unique():
    # select only the rows for the current id
    id_data_cov = data_cov[data_cov['id']==id]
    # Find index of the last observed value
    last_observed_index = id_data_cov['fast_insulin'].last_valid_index()
    # If there are no observed values at all, insert 0 to the last observation of the subset
    if pd.isna(last_observed_index):
        slow_insulin = id_data_cov['fast_insulin'].copy()
        id_data_cov.loc[id_data_cov.index[-1], 'fast_insulin'] = 0 
    else:
        # Insert 0 instead of NaN in slow_insulin after approx. 24 hours after last valis observation
        id_data_cov.loc[last_observed_index + 24, 'fast_insulin'] = 0
    # Append the subjects' subsets
    subset_data.append(id_data_cov)
# Covert the data back to dataframe  
data_cov = pd.concat(subset_data, ignore_index=True)

In [ ]:
## Interpolate insulin data

# Sort the data by subject ID and time
data = data_cov.sort_values(['id', 'time'])
interpolated_data = []

# Loop over each subject ID
for subject_id in data['id'].unique():
    # Select the rows for the current subject ID
    subject_data = data[data['id'] == subject_id]
    
    subject_data['slow_insulin'] = subject_data['slow_insulin'].interpolate(
        method='linear', limit_area = 'inside')
    subject_data['fast_insulin'] = subject_data['fast_insulin'].interpolate(
        method='linear', limit_area = 'inside')
    interpolated_data.append(subject_data)

data_cov = pd.concat(interpolated_data, ignore_index=True)
# Replace NaN with zeroes
data_cov = data_cov.fillna(0)

In [ ]:
# Plot the fast and slow insulin intakes for each subject
groups = data_cov.groupby('id')
num_cols = 3
num_rows = int(np.ceil(len(groups)/num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, (group_name, group) in enumerate(groups):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axs[row_idx, col_idx]
    ax.plot(group['time'], group['fast_insulin'], label='Fast Insulin')
    ax.plot(group['time'], group['slow_insulin'], label='Slow Insulin')
    ax.set_xlabel('Time')
    ax.set_ylabel('Insulin')
    ax.set_title(f'Insulin Plot for ID {group_name}')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Sum of NaN values in dataset
data_cov.isna().sum().sum()

In [ ]:
# Summary statistic of dataset
data_cov.describe()

## Food Data

In [ ]:
# Loop over the folder of each subject and merge files with insulin data by id
subject_ids = ["001", "002", "003", "004", "005", "006", "007", "008", "009"]

df_list = []
for subject_id in subject_ids:
    subject_data = pd.read_csv(f"raw_data/dubosson_covariates/diabetes_subset_pictures-glucose-food-insulin/{subject_id}/food.csv")
    subject_data["id"] = subject_id
    df_list.append(subject_data)

food_data = pd.concat(df_list, axis=0, ignore_index=True)
food_data

In [ ]:
# Drop Picture, Description columns
food_data.drop(["picture", "description"], axis=1, inplace=True)
# Covert subject ids to int64 to match with "data" ids
food_data['id'] = food_data['id'].astype(int)
# Check for NaNs
food_data.isna().sum() #present
# Drop rows with NaN values
food_data.dropna()
# drop rows with NaN values in datetime column
food_data = food_data.dropna(subset=['datetime'])
# drop rows with NaN values in balance column
food_data = food_data.dropna(subset=['balance'])
# drop rows with NaN values in quality column
food_data = food_data.dropna(subset=['quality'])
# Change the format of datetime column to match with glucose dataset
food_data['datetime'] = food_data['datetime'].apply(lambda x: datetime.datetime.strptime(x, '%Y:%m:%d %H:%M:%S'))


food_data

In [ ]:
# Get data summary
print(food_data['calories'].describe())
print(list(set(list(food_data['balance']))))
print(list(set(list(food_data['quality']))))

In [ ]:
# Recode the binary/categorical values
food_data['balance'] = food_data['balance'].replace({'Unbalance': 1, 'Balance': 2})
food_data['quality'] = food_data['quality'].replace({'Low quality': 1, 'Medium quality': 2, 'Good quality': 3})
food_data

In [ ]:
# Merge the two datasets based on "id"
df = data_cov.merge(food_data, on='id')
# For each row in insulin_data, calculate the absolute difference
df['diff'] = (df['datetime'] - df['time']).abs()
# Find the index of the minimum difference for each subject and each insulin date-time
idx = df.groupby(['id', 'datetime'])['diff'].idxmin()
# Use that index to retrieve the corresponding "time" value
df_final = df.loc[idx, ['id', 'datetime', 'time', 'calories', 'balance', 'quality']]
df_final.rename(columns={'id': 'id', 'time': 'closest_time'}, inplace=True)
# Add the closest time as a new column in insulin_data
result = data_cov.merge(df_final, left_on=['id', 'time'], right_on=['id', 'closest_time'], how='left')
# Calculate the difference between the closest time and datetime in minutes
result.loc[:, 'time_diff'] = np.abs((result['closest_time'] - result['datetime']) / np.timedelta64(1, 'm'))
# Keep only the rows where the absolute difference is less than or equal to 5 minutes
result = result.loc[result['time_diff'] <= 5, :]
result

In [ ]:
# Merge these duplicate rows in one row
result = result.groupby(["id", "closest_time"]).agg({"calories": "sum", "balance": "min", "quality": "min"}).reset_index()
# Merge glucose and insulin datasets
data_cov = data_cov.merge(result, how='left', left_on=['id', 'time'], right_on=['id', 'closest_time'])
# Drop closest_time column
data_cov.drop(["closest_time"], axis=1, inplace=True)
# Replace NaN with zerows
data_cov = data_cov.fillna(0)

data_cov

In [ ]:
# Loop through each subject and calculate the mean for that subject for balance and quality variables
for subject in data_cov['id'].unique():
    subject_data = data_cov[data_cov['id'] == subject]
    subject_mean = subject_data[subject_data[['balance', 'quality']] > 0][['balance', 'quality']].mean().round()
    if pd.isna(subject_mean).all():
        subject_mean = subject_mean.fillna(0)
    else:
        subject_mean = subject_mean.astype(int)
    
    # Replace the values in the original DataFrame with the subject-specific means
    data_cov.loc[data_cov['id'] == subject, 'balance'] = subject_mean['balance']
    data_cov.loc[data_cov['id'] == subject, 'quality'] = subject_mean['quality']

data_cov

In [ ]:
# Loop through subjects 5 and 9 and insert overall means for the food quality and balance, calories
subject_ids = [5, 9]
for subject in subject_ids:
    subject_data = data_cov[data_cov['id'] == subject]
    data_cov_mean = data_cov[data_cov[['balance', 'quality', 'calories']] > 0][['balance', 'quality', 'calories']].mean().round().astype(int)
    
    # Replace the NaN values in balance and quality columns with the subject-specific means
    data_cov.loc[(data_cov['id'] == subject), 'balance'] = data_cov_mean['balance']
    data_cov.loc[(data_cov['id'] == subject), 'quality'] = data_cov_mean['quality']
    data_cov.loc[(data_cov['id'] == subject), 'calories'] = data_cov_mean['calories']

data_cov

In [ ]:
## Interpolate calories data

#Replace NaN with zeroes
data_cov['calories'] = data_cov['calories'].replace(0, np.nan)

# set NaN right before observed values to 0 for calories
mask = data_cov['calories'].isna() & data_cov['calories'].shift(-1).notna()
data_cov.loc[mask, 'calories'] = 0

# Sort the data by subject ID and time
data = data_cov.sort_values(['id', 'time'])
interpolated_data = []

# Loop over each subject ID
for subject_id in data['id'].unique():
    # Select the rows for the current subject ID
    subject_data = data[data['id'] == subject_id]
    
    subject_data['calories'] = subject_data['calories'].interpolate(
        method='linear', limit_area = 'inside')
    interpolated_data.append(subject_data)

data_cov = pd.concat(interpolated_data, ignore_index=True)
# Replace NaN with zeroes
data_cov = data_cov.fillna(0)

data_cov

In [ ]:
# Plot the consumed food characteristics for each subject
groups = data_cov.groupby('id')
num_cols = 3
num_rows = int(np.ceil(len(groups)/num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, (group_name, group) in enumerate(groups):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axs[row_idx, col_idx]
    ax.plot(group['time'], group['balance'], label='Balance')
    ax.plot(group['time'], group['quality'], label='Quality')
    ax.set_xlabel('Time')
    ax.set_ylabel('Food characteristics')
    ax.set_title(f'Food characteristics Plot for ID {group_name}')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot the calories intakes for each subject
groups = data_cov.groupby('id')
num_cols = 3
num_rows = int(np.ceil(len(groups)/num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, (group_name, group) in enumerate(groups):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axs[row_idx, col_idx]
    ax.plot(group['time'], group['calories'])
    ax.set_xlabel('Time')
    ax.set_ylabel('Calories')
    ax.set_title(f'Calories Plot for ID {group_name}')

plt.tight_layout()
plt.show()

## Summary Statistic from wearable device covariates

In [ ]:
## Loop over the folder of each subject and merge files by id

# set the directory path
path = 'raw_data/dubosson_covariates/diabetes_subset_sensor_data'
# Create a list to store the dataframes for each subject
dfs = []

# Loop through each subject folder
for subject_id in ['001', '002', '003', '004', '005', '006', '007', '008', '009']:
    # Get the path to the sensor_data folder for the current subject
    sensor_data_path = os.path.join(path, subject_id, 'sensor_data')
    # Create a list to store the dataframes for each file in the current sensor_data folder
    files = []
    # Loop through each file in the current sensor_data folder
    sensor_dates = os.listdir(sensor_data_path)
    if '.DS_Store' in sensor_dates: sensor_dates.remove('.DS_Store')
    for filename in sensor_dates:
        # Get the path to the current file
        folderpath = os.path.join(sensor_data_path, filename)
        # Find the file with Summary statistic
        filepath = glob.glob(os.path.join(folderpath, '*Summary.csv'))
        # Read the data from the current file into a dataframe
        df = pd.read_csv(filepath[0])   
        # Add the subject_id to the dataframe
        df['id'] = subject_id 
        # Append the dataframe to the list of files
        files.append(df)
    # Concatenate the list of files into a single dataframe for the current subject
    subject_df = pd.concat(files)
    # Append the dataframe for the current subject to the list of dataframes
    dfs.append(subject_df)
# Concatenate the list of dataframes into a single dataframe for all subjects
summary_cov = pd.concat(dfs)

In [ ]:
# Drop rows with NaNs
summary_cov.dropna()
# Covert subject ids to int64 to match with "data" ids
summary_cov['id'] = summary_cov['id'].astype(int)
# Change the format of Time column to match with glucose dataset
summary_cov['Time'] = summary_cov['Time'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

summary_cov

In [ ]:
# Get the names of the columns
summary_cov.columns.values.tolist()

In [ ]:
# Summary statistics of the covariates
summary_cov.describe().applymap(lambda x: f"{x:0.3f}").transpose()

In [ ]:
# Heart Rate - beats per minute, range 25-240 (Invalid value = 65535)
# Breathing Rate - breaths per minute, range 3-70 (Invalid value = 6553.5)
# SkinTemp is not supported in this device (BioHarness 3.0 always returns 
#    an ‘Invalid’ value of -3276.8 °C for this parameter.)
# Posture - vertical = 0°, inverted = 180°(degrees), range +/- 180°
# Activity - range 16, (Invalid value = 655.35)
# BRAmplitude - Breathing Wave Amplitute (indicative only)
# ECGNoise - Breathing Wave Noise (indicative only)
# HRConfidence - Breathing Rate Confidence, % (Invalid value = 255)
# ECGAmplitude - ECG Amplitude (indicative only)
# ECGNoise - ECG Noise (indicative only)
# HRConfidence - Heart Rate Confidence, % (Invalid value = 255)
# HRV - HR Variability, range 0-280 (Invalid value = 65535)
# SystemConfidence - Physiological Data Validity, % (Invalid value = 255)
# StatusInfo - 16 bit number
# CoreTemp - Estimated Subject Core Temperature, range 33-41 (Invalid value = 6553.5)

## Keep only usefull covariates
summary_cov = summary_cov[['Time','HR','BR','Posture','Activity','HRV','SystemConfidence','CoreTemp','id']]

# drop rows where HR is outside range or has value of 65535
summary_cov = summary_cov[(summary_cov['HR'] >= 25) & (summary_cov['HR'] <= 240) & (summary_cov['HR'] != 65535)]
# drop rows where BR is outside range or has value of 6553.5
summary_cov = summary_cov[(summary_cov['BR'] >= 3) & (summary_cov['BR'] <= 70) & (summary_cov['BR'] != 6553.5)]
# drop rows where HRV is outside range or has value of 65535
summary_cov = summary_cov[(summary_cov['HRV'] >= 0) & (summary_cov['HRV'] <= 280) & (summary_cov['HRV'] != 65535)]
# drop rows where SystemConfidence is less than 50% or has value of 255
summary_cov = summary_cov[(summary_cov['SystemConfidence'] >= 50) & (summary_cov['SystemConfidence'] != 255)]
# drop rows where CoreTemp is outside range or has value of 6553.5
summary_cov = summary_cov[(summary_cov['CoreTemp'] >= 33) & (summary_cov['CoreTemp'] <= 41) & 
                          (summary_cov['CoreTemp'] != 6553.5)]
                                                                                
# Drop SystemConfidence and reset index
summary_cov = summary_cov.drop('SystemConfidence', axis=1)
summary_cov = summary_cov.reset_index(drop=True)
    
## Check the summary statistic again
summary_cov.describe().applymap(lambda x: f"{x:0.3f}").transpose()

In [ ]:
summary_cov

In [ ]:
## For the time grid from glucose data, calculate the average values of Summary Statistic from wearable device

# list of unique ids
ids = data_cov['id'].unique()
# Create a list to store the dataframes for each subject
dfs = []

# loop through each id
for id in ids:
    # subset data_cov and summary_cov by id
    data_cov_id = data_cov[data_cov['id'] == id].reset_index(drop=True)
    summary_cov_id = summary_cov[summary_cov['id'] == id].reset_index(drop=True)
    
    # loop through each time interval in data_cov
    for i in range(len(data_cov_id)-1):
        start_time = data_cov_id.loc[i, 'time']
        end_time = data_cov_id.loc[i+1, 'time'] 
        
        # subset summary_cov by time interval
        summary_cov_interval = summary_cov_id[(summary_cov_id['Time'] >= start_time) & 
                                              (summary_cov_id['Time'] < end_time)]
        
        # calculate average of other variables and fill in data_cov
        for col in summary_cov_interval.columns:
            if col not in ['Time', 'id']:
                avg_val = np.mean(summary_cov_interval[col])
                data_cov_id.loc[i, col] = avg_val
    
    # Calculate the average separately for the last timestamp
    if (i+1) == len(data_cov_id)-1:
        start_time = data_cov_id.loc[i+1, 'time']
        end_time = data_cov_id.loc[i+1, 'time'] + pd.Timedelta(minutes=5) # add 5 minutes to the last time stamp
        
        # subset summary_cov by time interval
        summary_cov_interval = summary_cov_id[(summary_cov_id['Time'] >= start_time) & 
                                              (summary_cov_id['Time'] < end_time)]
        
        # calculate average of other variables and fill in data_cov
        for col in summary_cov_interval.columns:
            if col not in ['Time', 'id']:
                avg_val = np.mean(summary_cov_interval[col])
                data_cov_id.loc[i+1, col] = avg_val
    
    # Append the dataframe for the current subject to the list of dataframes
    dfs.append(data_cov_id)
# Concatenate the list of dataframes into a single dataframe for all subjects
all_data = pd.concat(dfs)

all_data.reset_index(drop=True)

In [ ]:
# count number of NaN values per subject and column
nan_counts = all_data.groupby('id')['HR'].apply(lambda x: x.isna().sum())
# count number of observations per subject
obs_counts = all_data['id'].value_counts()
# calculate relative percentage of NaN values per subject
nan_percentages = nan_counts / obs_counts * 100
nan_percentages

In [ ]:
# #list of subject ids for which NaN values need to be replaced with 0
#subject_ids = [5, 9]
# #list of column names in which NaN values need to be replaced with 0
#columns_to_fill = ['HR', 'BR', 'Posture', 'Activity', 'HRV', 'CoreTemp']
# #replace NaN values with 0 for selected subjects and columns
#for subject_id in subject_ids:
#    for column in columns_to_fill:
#        all_data.loc[all_data['id'] == subject_id, column] = all_data.loc[all_data['id'] == subject_id, column].fillna(0)

#all_data

In [ ]:
# Plot the HR, BR, HRV, Posture, Activity, Core Temperature for each subject
groups = all_data.groupby('id')
num_cols = 3
num_rows = int(np.ceil(len(groups)/num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, (group_name, group) in enumerate(groups):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axs[row_idx, col_idx]
    ax.plot(group['time'], group['HR'], label='HR')
    ax.plot(group['time'], group['BR'], label='BR')
    ax.plot(group['time'], group['HRV'], label='HRV')
    ax.plot(group['time'], group['Posture'], label='Posture')
    ax.plot(group['time'], group['Activity'], label='Activity')
    ax.plot(group['time'], group['CoreTemp'], label='CoreTemp')
    ax.set_xlabel('Time')
    ax.set_ylabel('Insulin')
    ax.set_title(f'Heart and Breath Rates Plot for ID {group_name}')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
## Interpolate HR, BR, HRV data using Piecewise Cubic Hermite Interpolating Polynomial (PCHIP)

# Sort the data by subject ID and time
data = all_data.sort_values(['id', 'time'])
interpolated_data = []

# Loop over each subject ID
for subject_id in data['id'].unique():
    # Select the rows for the current subject ID
    subject_data = data[data['id'] == subject_id]
    
    subject_data['HR'] = subject_data['HR'].interpolate(method='pchip', limit_area = 'inside')
    subject_data['BR'] = subject_data['BR'].interpolate(method='pchip', limit_area = 'inside')
    subject_data['HRV'] = subject_data['HRV'].interpolate(method='pchip', limit_area = 'inside')
    subject_data['Posture'] = subject_data['Posture'].interpolate(method='pchip', limit_area = 'inside')
    subject_data['Activity'] = subject_data['Activity'].interpolate(method='pchip', limit_area = 'inside')
    subject_data['CoreTemp'] = subject_data['CoreTemp'].interpolate(method='pchip', limit_area = 'inside')
    interpolated_data.append(subject_data)

all_data = pd.concat(interpolated_data, ignore_index=True)

In [ ]:
# Plot the interpolated HR, BR, HRV, Posture, Activity, and Core Temperature data (from the inside) for each subject
groups = all_data.groupby('id')
num_cols = 3
num_rows = int(np.ceil(len(groups)/num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, (group_name, group) in enumerate(groups):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axs[row_idx, col_idx]
    ax.plot(group['time'], group['HR'], label='HR')
    ax.plot(group['time'], group['BR'], label='BR')
    ax.plot(group['time'], group['HRV'], label='HRV')
    ax.plot(group['time'], group['Posture'], label='Posture')
    ax.plot(group['time'], group['Activity'], label='Activity')
    ax.plot(group['time'], group['CoreTemp'], label='CoreTemp')
    ax.set_xlabel('Time')
    ax.set_ylabel('Insulin')
    ax.set_title(f'Heart and Breath Rates Plot for ID {group_name}')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Loop through each subject and calculate the mean for the wearable device variables to fill the outside NaN values
for subject in all_data['id'].unique():
    subject_data = all_data[all_data['id'] == subject]
    subject_mean = subject_data[['HR', 'BR', 'HRV', 'Posture', 'Activity', 'CoreTemp']].mean()
    
    # Replace the NaN values in balance and quality columns with the subject-specific means
    all_data.loc[(all_data['id'] == subject) & (all_data['HR'].isna()), 'HR'] = subject_mean['HR']
    all_data.loc[(all_data['id'] == subject) & (all_data['BR'].isna()), 'BR'] = subject_mean['BR']
    all_data.loc[(all_data['id'] == subject) & (all_data['HRV'].isna()), 'HRV'] = subject_mean['HRV']
    all_data.loc[(all_data['id'] == subject) & (all_data['Posture'].isna()), 'Posture'] = subject_mean['Posture']
    all_data.loc[(all_data['id'] == subject) & (all_data['Activity'].isna()), 'Activity'] = subject_mean['Activity']
    all_data.loc[(all_data['id'] == subject) & (all_data['CoreTemp'].isna()), 'CoreTemp'] = subject_mean['CoreTemp']

all_data

In [ ]:
# Loop through subjects 5 and 9 and insert overall means for the wearable device covariates
subject_ids = [5, 9]
for subject in subject_ids:
    subject_data = all_data[all_data['id'] == subject]
    all_data_mean = all_data[['HR', 'BR', 'HRV', 'Posture', 'Activity', 'CoreTemp']].mean()
    
    # Replace the NaN values in balance and quality columns with the subject-specific means
    all_data.loc[(all_data['id'] == subject) & (all_data['HR'].isna()), 'HR'] = all_data_mean['HR']
    all_data.loc[(all_data['id'] == subject) & (all_data['BR'].isna()), 'BR'] = all_data_mean['BR']
    all_data.loc[(all_data['id'] == subject) & (all_data['HRV'].isna()), 'HRV'] = all_data_mean['HRV']
    all_data.loc[(all_data['id'] == subject) & (all_data['Posture'].isna()), 'Posture'] = all_data_mean['Posture']
    all_data.loc[(all_data['id'] == subject) & (all_data['Activity'].isna()), 'Activity'] = all_data_mean['Activity']
    all_data.loc[(all_data['id'] == subject) & (all_data['CoreTemp'].isna()), 'CoreTemp'] = all_data_mean['CoreTemp']

all_data

In [ ]:
# Plot the fully interpolated HR, BR, HRV, Posture, Activity, and Core Temperature data for each subject
groups = all_data.groupby('id')
num_cols = 3
num_rows = int(np.ceil(len(groups)/num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, (group_name, group) in enumerate(groups):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axs[row_idx, col_idx]
    ax.plot(group['time'], group['HR'], label='HR')
    ax.plot(group['time'], group['BR'], label='BR')
    ax.plot(group['time'], group['HRV'], label='HRV')
    ax.plot(group['time'], group['Posture'], label='Posture')
    ax.plot(group['time'], group['Activity'], label='Activity')
    ax.plot(group['time'], group['CoreTemp'], label='CoreTemp')
    ax.set_xlabel('Time')
    ax.set_ylabel('Insulin')
    ax.set_title(f'Heart and Breath Rates Plot for ID {group_name}')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
all_data.isna().sum() # no NaN values anymore

In [ ]:
# save as Dubosson_processed_with_covariates.csv
all_data.to_csv('./raw_data/Dubosson_processed_with_covariates.csv', index=False)

# Check statistics of the data

In [ ]:
import matplotlib.pyplot as plt

# load yaml config file
with open('./config/dubosson.yaml', 'r') as f:
    config = yaml.safe_load(f)

# set interpolation params for no interpolation
new_config = config.copy()
new_config['interpolation_params']['gap_threshold'] = 5
new_config['interpolation_params']['min_drop_length'] = 0
# set split params for no splitting
new_config['split_params']['test_percent_subjects'] = 0
new_config['split_params']['length_segment'] = 0
# set scaling params for no scaling
new_config['scaling_params']['scaler'] = 'None'

formatter = DataFormatter(new_config)

In [ ]:
# loop through each column in the dataset
for col in formatter.train_data.columns:
    if col in ['id_segment', 'time', 'id', 'time_year', 'time_month', 'time_day', 'time_hour', 'time_minute']:
        continue
    col_data = formatter.train_data[col]
    
    # print summary statistics
    print('{} column:'.format(col))
    print('\tMin: ', col_data.min())
    print('\tMax: ', col_data.max())
    print('\t25th percentile: ', np.percentile(col_data, 25))
    print('\tMedian: ', col_data.median())
    print('\t75th percentile: ', np.percentile(col_data, 75))
    print('\tMean: ', col_data.mean())
    print('\tStd: ', col_data.std())

    # plot data for each segment
    num_segments = formatter.train_data['id_segment'].nunique()
    fig, axs = plt.subplots(1, num_segments, figsize=(30, 5))
    for i, (group, data) in enumerate(formatter.train_data.groupby('id_segment')):
        data.plot(x='time', y=col, ax=axs[i], title='{} Segment {}'.format(col, group))


In [ ]:
# plot acf of random samples from segments
fig, ax = plt.subplots(2, num_segments, figsize=(30, 5))
lags = 300
for i, (group, data) in enumerate(formatter.train_data.groupby('id_segment')):
    data = data['gl']
    if len(data) < lags:
        print('Segment {} is too short'.format(group))
        continue
    # select 10 random samples from index of data
    sample = np.random.choice(range(len(data))[:-lags], 10, replace=False)
    # plot acf / pacf of each sample
    for j in sample:
        acf, acf_ci = sm.tsa.stattools.acf(data[j:j+lags], nlags=lags, alpha=0.05)
        pacf, pacf_ci = sm.tsa.stattools.pacf(data[j:j+lags], method='ols-adjusted', alpha=0.05)
        ax[0, i].plot(acf)
        ax[1, i].plot(pacf)

# Change the config according to the observations above

In [ ]:
with open('./config/dubosson.yaml', 'r') as f:
    config = yaml.safe_load(f)

# set interpolation params for no interpolation
config['interpolation_params']['gap_threshold'] = 30
config['interpolation_params']['min_drop_length'] = 240
# set split params for no splitting
config['split_params']['test_percent_subjects'] = 0.1
config['split_params']['length_segment'] = 240
# set scaling params for no scaling
config['scaling_params']['scaler'] = 'None'

formatter = DataFormatter(config)

# Models (no covariates)

## Convert data and (optional) scaling

In [ ]:
def load_data(seed = 0, study_file = None):
    # load data
    with open('./config/dubosson.yaml', 'r') as f:
        config = yaml.safe_load(f)
    config['split_params']['random_state'] = seed
    formatter = DataFormatter(config, study_file = study_file)

    # convert to series
    time_col = formatter.get_column('time')
    group_col = formatter.get_column('sid')
    target_col = formatter.get_column('target')
    static_cols = formatter.get_column('static_covs')
    static_cols = static_cols + [formatter.get_column('id')] if static_cols is not None else [formatter.get_column('id')]
    dynamic_cols = formatter.get_column('dynamic_covs')
    future_cols = formatter.get_column('future_covs')

    # build series
    series, scalers = make_series({'train': formatter.train_data,
                                    'val': formatter.val_data,
                                    'test': formatter.test_data.loc[~formatter.test_data.index.isin(formatter.test_idx_ood)],
                                    'test_ood': formatter.test_data.loc[formatter.test_data.index.isin(formatter.test_idx_ood)]},
                                    time_col,
                                    group_col,
                                    {'target': target_col,
                                    'static': static_cols,
                                    'dynamic': dynamic_cols,
                                    'future': future_cols})
    
    return formatter, series, scalers

formatter, series, scalers = load_data()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 3))
hours = []
for i in range(len(series['test']['target'])):
    hours += list(series['test']['target'][i].time_index[formatter.params['max_length_input']: ].hour)
axs[0].hist(hours, bins=36)

hours = []
for i in range(len(series['test_ood']['target'])):
    hours += list(series['test_ood']['target'][i].time_index[formatter.params['max_length_input']: ].hour)
axs[1].hist(hours, bins=36)

In [ ]:
for f in series['train']['target']:
    f.plot(label='train', alpha=0.1, color='grey')
series['test_ood']['target'][0].plot(label='test_ood')
for f in series['test']['target']:
    f.plot(label='test_id', color='orange', alpha=0.5)

# ARIMA

In [ ]:
def test_model(test_data, scaler, in_len, out_len, stride, target_col, group_col):
    errors = []
    for group, data in test_data.groupby(group_col):
        train_set = data[target_col].iloc[:in_len].values.flatten()
        # fit model
        model = AutoARIMA(start_p = 0,
                        max_p = 10,
                        start_q = 0,
                        max_q = 10,
                        start_P = 0,
                        max_P = 10,
                        start_Q=0,
                        max_Q=10,
                        allowdrift=True,
                        allowmean=True,
                        parallel=False)
        model.fit(train_set)
        # get valid sampling locations for future prediction
        start_idx = np.arange(start=stride, stop=len(data) - in_len - out_len + 1, step=stride)
        end_idx = start_idx + in_len
        # iterate and collect predictions
        for i in range(len(start_idx)):
            input = data[target_col].iloc[start_idx[i]:end_idx[i]].values.flatten()
            true = data[target_col].iloc[end_idx[i]:(end_idx[i]+out_len)].values.flatten()
            prediction = model.forward(input, h=out_len)['mean']
            # unscale true and prediction
            true = scaler.inverse_transform(true.reshape(-1, 1)).flatten()
            prediction = scaler.inverse_transform(prediction.reshape(-1, 1)).flatten()
            # collect errors
            errors.append(np.array([np.mean((true - prediction)**2), np.mean(np.abs(true - prediction))]))
    errors = np.vstack(errors)
    return errors

In [ ]:
# load data
with open('./config/dubosson.yaml', 'r') as f:
        config = yaml.safe_load(f)
config['split_params']['random_state'] = 0
config['scaling_params']['scaler'] = 'StandardScaler'
formatter = DataFormatter(config, study_file = None)

# set params
in_len = formatter.params['max_length_input']
out_len = formatter.params['length_pred']
stride = formatter.params['length_pred'] // 2
target_col = formatter.get_column('target')
group_col = formatter.get_column('sid')

In [ ]:
test_data = formatter.test_data.loc[~formatter.test_data.index.isin(formatter.test_idx_ood)]
test_data_ood = formatter.test_data.loc[formatter.test_data.index.isin(formatter.test_idx_ood)]

# backtest on the ID test set
id_errors_sample = test_model(test_data, formatter.scalers[target_col[0]], in_len, out_len, stride, target_col, group_col)
id_errors_sample = np.vstack(id_errors_sample)
id_error_stats_sample = compute_error_statistics(id_errors_sample)

# backtest on the ood test set
ood_errors_sample = test_model(test_data_ood, formatter.scalers[target_col[0]], in_len, out_len, stride, target_col, group_col)
ood_errors_sample = np.vstack(ood_errors_sample)
ood_errors_stats_sample = compute_error_statistics(ood_errors_sample)

## Linear regression

In [ ]:
model = models.LinearRegressionModel(lags = 60,
                                     output_chunk_length = formatter.params['length_pred'])

model.fit(series['train']['target'],
          max_samples_per_ts=None, 
          )

In [ ]:
forecasts = model.historical_forecasts(series['test']['target'],
                                        forecast_horizon=formatter.params['length_pred'], 
                                        stride=formatter.params['length_pred'] // 2,
                                        retrain=False,
                                        verbose=True,
                                        last_points_only=False,
                                        start=formatter.params['max_length_input'])
id_errors_sample = rescale_and_backtest(series['test']['target'],
                                    forecasts,  
                                    [metrics.mse, metrics.mae],
                                    scalers['target'],
                                    reduction=None)
id_errors_sample = np.vstack(id_errors_sample)
np.median(id_errors_sample, axis=0)

In [ ]:
forecasts = model.historical_forecasts(series['test_ood']['target'],
                                                forecast_horizon=formatter.params['length_pred'], 
                                                stride=formatter.params['length_pred'] // 2,
                                                retrain=False,
                                                verbose=True,
                                                last_points_only=False,
                                                start=formatter.params["max_length_input"])
ood_errors_sample = rescale_and_backtest(series['test_ood']['target'],
                            forecasts,  
                            [metrics.mse, metrics.mae],
                            scalers['target'],
                            reduction=None)
ood_errors_sample = np.vstack(ood_errors_sample)
np.median(ood_errors_sample, axis=0)

## XGBoost

In [ ]:
model = models.XGBModel(lags=96, 
                        learning_rate=0.773,
                        subsample=0.8,
                        min_child_weight=1.0,
                        colsample_bytree=1.0,
                        max_depth=6,
                        gamma=0.5,
                        reg_alpha=0.167,
                        reg_lambda=0.229,
                        n_estimators=352,
                        model_seed=0)

model.fit(series['train']['target'],
          max_samples_per_ts=None, 
          )

In [ ]:
forecasts = model.historical_forecasts(series['test']['target'],
                                        forecast_horizon=formatter.params['length_pred'], 
                                        stride=formatter.params['length_pred'] // 2,
                                        retrain=False,
                                        verbose=True,
                                        last_points_only=False,
                                        start=formatter.params['max_length_input'])
id_errors_sample = rescale_and_backtest(series['test']['target'],
                                    forecasts,  
                                    [metrics.mse, metrics.mae],
                                    scalers['target'],
                                    reduction=None)
id_errors_sample = np.vstack(id_errors_sample)
np.median(id_errors_sample, axis=0)

In [ ]:
forecasts = model.historical_forecasts(series['test_ood']['target'],
                                                forecast_horizon=formatter.params['length_pred'], 
                                                stride=formatter.params['length_pred'] // 2,
                                                retrain=False,
                                                verbose=True,
                                                last_points_only=False,
                                                start=formatter.params["max_length_input"])
ood_errors_sample = rescale_and_backtest(series['test_ood']['target'],
                            forecasts,  
                            [metrics.mse, metrics.mae],
                            scalers['target'],
                            reduction=None)
ood_errors_sample = np.vstack(ood_errors_sample)
np.median(ood_errors_sample, axis=0)

In [ ]:
def rescale_and_backtest(series: Union[TimeSeries, Sequence[TimeSeries]],
                         forecasts: Union[TimeSeries, Sequence[TimeSeries]], 
                         metric: Union[
                                    Callable[[TimeSeries, TimeSeries], float],
                                    List[Callable[[TimeSeries, TimeSeries], float]],
                                ], 
                         scaler: Callable[[TimeSeries], TimeSeries] = None,
                         reduction: Union[Callable[[np.ndarray], float], None] = np.mean,
                        ):
    """
    Backtest the forecasts on the series.

    Parameters
    ----------
    series
        The target time series.
    forecasts
        The forecasts.
    scaler
        The scaler used to scale the series.
    metric
        The metric to use for backtesting.
    reduction
        The reduction to apply to the metric.

    Returns
    -------
    float or List[float] or List[List[float]]
        The (sequence of) error score on a series, or list of list containing error scores for each
        provided series and each sample.
    """
    series = [series] if isinstance(series, TimeSeries) else series
    if len(series) == 1:
        forecasts = [forecasts]
    if not isinstance(metric, list):
        metric = [metric]

    # reverse scaling, forecasts and true values, compute errors
    backtest_list = []
    for idx, target_ts in enumerate(series):
        if scaler is not None:
            target_ts = scaler.inverse_transform(target_ts)
            predicted_ts = [scaler.inverse_transform(f) for f in forecasts[idx]]
        errors = [
            [metric_f(target_ts, f) for metric_f in metric]
            if len(metric) > 1
            else metric[0](target_ts, f)
            for f in predicted_ts
            if f.time_index.hour[0] > 6 and f.time_index.hour[0] < 18
        ]
        if reduction is None:
            backtest_list.append(np.array(errors))
        else:
            backtest_list.append(reduction(np.array(errors), axis=0))
    return backtest_list if len(backtest_list) > 1 else backtest_list[0]

ood_errors_sample = rescale_and_backtest(series['test_ood']['target'],
                            forecasts,  
                            [metrics.mse, metrics.mae],
                            scalers['target'],
                            reduction=None)
ood_errors_sample = np.vstack(ood_errors_sample)
np.median(ood_errors_sample, axis=0)

In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(50, 20))
for i in range(6):
    for j, f in enumerate(forecasts[i][:5]):
        true = scalers['target'].inverse_transform(series['test']['target'][i].slice_n_points_after(f.time_index[0] - pd.Timedelta("2h"), 36))
        forecast = scalers['target'].inverse_transform(f)
        forecast.plot(ax=axs[j, i], label='forecast')
        true.plot(ax=axs[j, i], label='true')
        axs[j, i].legend(fontsize=14)

In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(50, 20))
for i in range(6):
    for j in range(5):
        forecast = scalers['target'].inverse_transform(forecasts[i+6*j])
        true = scalers['target'].inverse_transform(series['test_ood']['target'][0].slice_n_points_after(forecast.time_index[0] - pd.Timedelta("2h"), 36))
        forecast.plot(ax=axs[j, i], label='forecast')
        true.plot(ax=axs[j, i], label='true')

## TFT

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

model_name = f'tensorboard_tft_weinstock'
work_dir = './output'
el_stopper = EarlyStopping(
                            monitor="val_loss",
                            patience=20,
                            min_delta=0.001,
                            mode='min',
                            )
loss_logger = LossLogger()
pl_trainer_kwargs = {"accelerator": "gpu", "devices": [0], "callbacks": [el_stopper, loss_logger]}

# build the TFTModel model
model = models.TFTModel(input_chunk_length = 96, 
                        output_chunk_length = formatter.params['length_pred'], 
                        hidden_size = 64,
                        lstm_layers = 1,
                        num_attention_heads = 4,
                        full_attention = True,
                        dropout = 0.1,
                        hidden_continuous_size = 16,
                        add_relative_index = True,
                        model_name = model_name,
                        work_dir = work_dir,
                        log_tensorboard = True,
                        pl_trainer_kwargs = pl_trainer_kwargs,
                        batch_size = 64,
                        optimizer_kwargs = {'lr': 0.001},
                        save_checkpoints = True,
                        force_reset=True)

In [ ]:
model.fit(series=series['train']['target'],
              val_series=series['val']['target'],
              max_samples_per_ts=200,
              verbose=True,)

In [ ]:
forecasts = model.historical_forecasts(series['test']['target'],
                                        forecast_horizon=formatter.params['length_pred'], 
                                        stride=formatter.params['length_pred'] // 2,
                                        retrain=False,
                                        verbose=False,
                                        last_points_only=False,
                                        start=formatter.params['max_length_input'])

In [ ]:
errors = rescale_and_backtest(series['test']['target'],
                                      forecasts,  
                                      [metrics.mse, metrics.mae],
                                      scalers['target'],
                                      reduction=None)
errors = np.vstack(errors)         
np.median(errors, axis=0)                             

In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(50, 20))
for i in range(6):
    for j, f in enumerate(forecasts[i][:5]):
        f.plot(ax=axs[j, i], label='forecast')
        series['test']['target'][i].slice_n_points_after(f.time_index[0] - pd.Timedelta("2h"), 36).plot(ax=axs[j, i], label='true')
        axs[j, i].legend(fontsize=14)

## NHiTS

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

model_name = f'tensorboard_nhits_weinstock'
work_dir = './output'
el_stopper = EarlyStopping(monitor="val_loss", patience=20, min_delta=0.001, mode='min')
loss_logger = LossLogger()
pl_trainer_kwargs = {"accelerator": "gpu", "devices": [0], "callbacks": [el_stopper, loss_logger]}

# build the TFTModel model
model = models.NHiTSModel(input_chunk_length=96, 
                            output_chunk_length=12, 
                            num_stacks=3, 
                            num_blocks=1, 
                            num_layers=2, 
                            layer_widths=512,  
                            n_freq_downsample=None, 
                            dropout=0.05, 
                            activation='ReLU',
                            log_tensorboard = True,
                            pl_trainer_kwargs = pl_trainer_kwargs,
                            batch_size = 64,
                            optimizer_kwargs = {'lr': 0.001},
                            save_checkpoints = True,
                            model_name = model_name,
                            work_dir = work_dir,
                            force_reset=True)

In [ ]:
model.fit(series=series['train']['target'],
              val_series=series['val']['target'],
              max_samples_per_ts=200,
              verbose=True,)

In [ ]:
model.load_from_checkpoint(model_name, work_dir = work_dir)

forecasts = model.historical_forecasts(series['test']['target'],
                                        forecast_horizon=formatter.params['length_pred'], 
                                        stride=formatter.params['length_pred'] // 2,
                                        retrain=False,
                                        verbose=False,
                                        last_points_only=False,
                                        start=formatter.params['max_length_input'])

errors = rescale_and_backtest(series['test']['target'],
                                      forecasts,  
                                      [metrics.mse, metrics.mae],
                                      scalers['target'],
                                      reduction=None)
errors = np.vstack(errors)         
np.median(errors, axis=0)                             

In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(50, 20))
for i in range(6):
    for j, f in enumerate(forecasts[i][:5]):
        f.plot(ax=axs[j, i], label='forecast')
        series['test']['target'][i].slice_n_points_after(f.time_index[0] - pd.Timedelta("2h"), 36).plot(ax=axs[j, i], label='true')
        axs[j, i].legend(fontsize=14)

# Model (with covariates)

## Convert data and optional scaling

In [ ]:
def load_data(seed = 0, study_file = None):
    # load data
    with open('./config/dubosson.yaml', 'r') as f:
        config = yaml.safe_load(f)
    config['split_params']['random_state'] = seed
    formatter = DataFormatter(config, study_file = study_file)

    # convert to series
    time_col = formatter.get_column('time')
    group_col = formatter.get_column('sid')
    target_col = formatter.get_column('target')
    static_cols = formatter.get_column('static_covs')
    static_cols = static_cols + [formatter.get_column('id')] if static_cols is not None else [formatter.get_column('id')]
    dynamic_cols = formatter.get_column('dynamic_covs')
    future_cols = formatter.get_column('future_covs')

    # build series
    series, scalers = make_series({'train': formatter.train_data,
                                    'val': formatter.val_data,
                                    'test': formatter.test_data.loc[~formatter.test_data.index.isin(formatter.test_idx_ood)],
                                    'test_ood': formatter.test_data.loc[formatter.test_data.index.isin(formatter.test_idx_ood)]},
                                    time_col,
                                    group_col,
                                    {'target': target_col,
                                    'static': static_cols,
                                    'dynamic': dynamic_cols,
                                    'future': future_cols})
    
    # attach static covariates to series
    for i in range(len(series['train']['target'])):
        static_covs = series['train']['static'][i][0].pd_dataframe()
        series['train']['target'][i] = series['train']['target'][i].with_static_covariates(static_covs)
    # attach to validation and test series
    for i in range(len(series['val']['target'])):
        static_covs = series['val']['static'][i][0].pd_dataframe()
        series['val']['target'][i] = series['val']['target'][i].with_static_covariates(static_covs)
    for i in range(len(series['test']['target'])):
        static_covs = series['test']['static'][i][0].pd_dataframe()
        series['test']['target'][i] = series['test']['target'][i].with_static_covariates(static_covs)
    
    return formatter, series, scalers

formatter, series, scalers = load_data()

In [ ]:
static_cols = formatter.get_column('static_covs')
static_cols = static_cols + [formatter.get_column('id')] if static_cols is not None else [formatter.get_column('id')]
dynamic_cols = formatter.get_column('dynamic_covs')
future_cols = formatter.get_column('future_covs')
# convert None to empty list
static_cols = [] if static_cols is None else static_cols
dynamic_cols = [] if dynamic_cols is None else dynamic_cols
future_cols = [] if future_cols is None else future_cols
# concatenate all covariates
covs = static_cols + dynamic_cols + future_cols

train_data = formatter.train_data
test_data = formatter.test_data.loc[~formatter.test_data.index.isin(formatter.test_idx_ood)]
test_data_ood = formatter.test_data.loc[formatter.test_data.index.isin(formatter.test_idx_ood)]

# plot histograms and densities of covariates using seaborn
fig, axs = plt.subplots(len(covs) // 3 + 1, 3, figsize=(10, 45))
for i, c in enumerate(covs):
    # create a dataframe with covariate and type
    df = pd.DataFrame({'covariate': train_data[c].values, 'type': 'train'})
    df2 = pd.DataFrame({'covariate': test_data[c].values, 'type': 'test'})
    df3 = pd.DataFrame({'covariate': test_data_ood[c].values, 'type': 'test_ood'})
    df = pd.concat([df, df2, df3])
    # plot density
    sns.histplot(data=df, 
                 x='covariate', 
                 hue='type', 
                 ax=axs[i // 3, i % 3], 
                 stat='density', 
                 common_norm=False, 
                 common_bins=True,
                 multiple='fill')
    axs[i // 3, i % 3].set_title(c)
plt.tight_layout()


## Linear Regression

In [ ]:
for i in range(len(series['train']['target'])):
    assert len(series['train']['future'][i]) == len(series['train']['target'][i])

In [ ]:
model = models.LinearRegressionModel(lags = 192,
                                     lags_future_covariates = (192, 12),
                                     output_chunk_length = formatter.params['length_pred'])

model.fit(series['train']['target'],
          future_covariates=series['train']['future'],
          max_samples_per_ts=100)

In [ ]:
forecasts = model.historical_forecasts(series['test']['target'],
                                        future_covariates=series['test']['future'],
                                        forecast_horizon=formatter.params['length_pred'], 
                                        stride=formatter.params['length_pred'] // 2,
                                        retrain=False,
                                        verbose=True,
                                        last_points_only=False,
                                        start=formatter.params['max_length_input'])
id_errors_sample = rescale_and_backtest(series['test']['target'],
                                    forecasts,  
                                    [metrics.mse, metrics.mae],
                                    scalers['target'],
                                    reduction=None)
id_errors_sample = np.vstack(id_errors_sample)
np.median(id_errors_sample, axis=0)

In [ ]:
forecasts = model.historical_forecasts(series['test_ood']['target'],
                                        future_covariates=series['test_ood']['future'],
                                        forecast_horizon=formatter.params['length_pred'], 
                                        stride=formatter.params['length_pred'] // 2,
                                        retrain=False,
                                        verbose=True,
                                        last_points_only=False,
                                        start=formatter.params["max_length_input"])
ood_errors_sample = rescale_and_backtest(series['test_ood']['target'],
                            forecasts,  
                            [metrics.mse, metrics.mae],
                            scalers['target'],
                            reduction=None)
ood_errors_sample = np.vstack(ood_errors_sample)
np.median(ood_errors_sample, axis=0)

In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(50, 20))
for i in range(6):
    for j, f in enumerate(forecasts[i][:5]):
        f.plot(axs[j, i], label='forecast')
        series['val']['target'][i].slice_n_points_after(f.time_index[0] - pd.Timedelta("2h"), 36).plot(axs[j, i], label='true')
        axs[j, i].legend(fontsize=14)